This project operates on public Central Statistical Office and was made only for learning purposes.

Downloaded informations **were not used for any further processing**.

#### Importing libraries

In [1]:
import pandas as pd
import numpy as np

# Selenium libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
#ignoring this exceptions in main loop will allow browser to wait until elements appear on site.
ignored_exceptions = (NoSuchElementException,StaleElementReferenceException,) 

#Other libraries
import time
from datetime import datetime
import winsound

#### Reading data

In [2]:
NIP = pd.read_csv('Data/State_100_share_companies.csv')
NIP.head()

,NIP
0,899-232-12-94
1,526-267-51-21
2,701-030-31-02
3,701-030-24-56
4,657-008-13-17


Converting NIP numbers to list

In [3]:
NIP_list = list(NIP['NIP'])

In [4]:
print(f'There are {len(NIP_list)} companies to search.')

There are 143 companies to search.


#### Creating empty dataframe with desired column names.

In [5]:
columns = ['REGON','NIP','NIP_status','name','code_and_name_of_the_primary_legal_form',
           'code_and_name_of_the_specific_legal_form', 'code_and_name_of_the_ownership_form',
           'registration_authority', 'type_of_register_or_records', 'number_in_register_or_records',
           'country','voivodeship', 'county', 'commune', 'city', 'street', 'property_number',
           'apartment_number', 'postcode', 'postal_city', 'unusual_location', 'date_of_entry_in_the_register_or_records',
           'date_of_origin','date_of_commencement_of_activities', 'date_of_entry_to_REGON', 
           'date_of_activity_suspension', 'date_of_activity_resumption', 'date_of_activity_cessation',
           'date_of_deletion_from_REGON', 'date_of_the_judgment_about_bankruptcy_declaration',
           'date_of_insolvency_proceedings_ending', 'phone number', 'internal_phone_number','fax_number',
           'email', 'website_address', 'PKD_code','PKD_name'
          ]
df = pd.DataFrame(columns = columns)
df

,REGON,NIP,NIP_status,name,code_and_name_of_the_primary_legal_form,code_and_name_of_the_specific_legal_form,code_and_name_of_the_ownership_form,registration_authority,type_of_register_or_records,number_in_register_or_records,...,date_of_deletion_from_REGON,date_of_the_judgment_about_bankruptcy_declaration,date_of_insolvency_proceedings_ending,phone number,internal_phone_number,fax_number,email,website_address,PKD_code,PKD_name


#### Creating webdriver and loading website

In [6]:
driver = webdriver.Chrome()
driver.implicitly_wait(1)
driver.get('https://wyszukiwarkaregon.stat.gov.pl/')
driver.implicitly_wait(1)

#### Searching nip numbers on website and saving informations to dataframe.

In [7]:
start_time = datetime.now()

failed_NIP_list = [] # list for companies that weren't added to dataframe.

for index, nip_number in enumerate(NIP_list):
    
    # For each nip number, program makes up to 10 attempts to complete company information.
    # During fail it refreshes the page, wait 2 seconds and tries again.
    attempts = 0
    while attempts < 10:
        try:
            company_information = [] # list of informations about currently analized company
            
            # Waiting untill search box becomes available
            WebDriverWait(driver, 5,ignored_exceptions=ignored_exceptions)\
                        .until(EC.presence_of_element_located((By.ID, 'txtNip'))) 
            
            elem = driver.find_element(By.ID,'txtNip')  # Find the NIP search box
            elem.clear() # Clear any text in that search box
            elem.send_keys(nip_number + Keys.ENTER) # Send current nip number into search box and press enter
            
            #locating div with informations when it becomes available.
            my_div = WebDriverWait(driver, 5,ignored_exceptions=ignored_exceptions)\
                        .until(EC.presence_of_element_located((By.ID, 'divListaJednostek'))) 
            
            #Clicking link inside div, that load main table on website.
            WebDriverWait(my_div, 5,ignored_exceptions=ignored_exceptions)\
                        .until(EC.element_to_be_clickable((By.TAG_NAME, 'a'))) 
            driver.find_element(By.ID, 'divListaJednostek').find_element(By.TAG_NAME, 'a').click()

            # Locating all available tables with company informations.
            try:
                tables = driver.find_elements(By.CSS_SELECTOR,'div.left.divGroupFour')
            except:
                tables = driver.find_elements(By.CSS_SELECTOR,'div.left.divGroup')

            # Saving informations from located tables
            for table in tables:
                for row in table.find_elements(By.CSS_SELECTOR,'tr')[1:]:
                    cells = row.find_elements(By.CSS_SELECTOR,'td')
                    if cells:
                        company_information.append(cells[-1].text)

            # Clicking link that load additional table with PKD informations
            try:
                driver.find_element(By.CSS_SELECTOR,"div[id='praw_linkdzial']").click()
            except:
                driver.find_element(By.CSS_SELECTOR, "div[id='fiz_linkdzial']").click()
            # Locating loaded table with PKD informations
            try:
                WebDriverWait(driver, 5,ignored_exceptions=ignored_exceptions)\
                        .until(EC.presence_of_element_located((By.CSS_SELECTOR,'table.tabelaZbiorcza.tabelaZbiorczaPKD'))) 
                table = driver.find_element(By.CSS_SELECTOR,'table.tabelaZbiorcza.tabelaZbiorczaPKD')
            except:
                WebDriverWait(driver, 5,ignored_exceptions=ignored_exceptions)\
                        .until(EC.presence_of_element_located((By.CSS_SELECTOR,'table.tabelaZbiorcza'))) 
                table = driver.find_element(By.CSS_SELECTOR,'table.tabelaZbiorcza')
            # Saving main PKD code and name
            row_with_pkd_info = table.find_elements(By.CSS_SELECTOR,'tr')[1] #Locating row with main pkd code and name
            pkd_code = row_with_pkd_info.find_elements(By.CSS_SELECTOR,'td')[0].text
            pkd_name = row_with_pkd_info.find_elements(By.CSS_SELECTOR,'td')[1].text
            company_information += [pkd_code,pkd_name]       
            
            # Saving found informations about company as a new row in dataframe
            df.loc[len(df)] = company_information
            print(f'{index+1}. {nip_number} information was succesfully saved')  
            break
        except:
            attempts += 1
            if attempts == 10:
                failed_NIP_list.append(nip_number)
            print(f'{nip_number} fail number: {attempts}')
            driver.get('https://wyszukiwarkaregon.stat.gov.pl/')
            time.sleep(2)
            
end_time = datetime.now()
driver.quit()
winsound.Beep(440, 1000) # Sound effect to inform about the finished work

1. 899-232-12-94 information was succesfully saved
2. 526-267-51-21 information was succesfully saved
3. 701-030-31-02 information was succesfully saved
4. 701-030-24-56 information was succesfully saved
5. 657-008-13-17 information was succesfully saved
6. 773-000-92-24 information was succesfully saved
7. 649-000-04-00 information was succesfully saved
8. 564-000-16-14 information was succesfully saved
9. 525-100-84-09 information was succesfully saved
10. 882-000-28-35 information was succesfully saved
11. 816-148-33-09 information was succesfully saved
12. 526-102-75-26 information was succesfully saved
13. 627-001-16-37 information was succesfully saved
14. 525-000-12-27 information was succesfully saved
15. 694-000-35-46 information was succesfully saved
16. 525-100-04-54 information was succesfully saved
17. 701-009-35-15 information was succesfully saved
18. 525-000-43-78 information was succesfully saved
19. 716-000-18-74 information was succesfully saved
20. 784-000-34-35 inf

In [8]:
print(f"Done in {pd.to_timedelta(end_time-start_time).round('1s')}")

Done in 0 days 00:11:35


#### Checking if every company informations were added to dataframe

In [9]:
if (len(df) == len(NIP_list)) and not failed_NIP_list:
    print('Got informations for all companies!')
else:
    print("Warning: Couldn't add some companies information to dataframe! Fails are stored in 'failed_NIP_list' variable.")

Got informations for all companies!


#### Dataframe after main loop.

In [10]:
df

,REGON,NIP,NIP_status,name,code_and_name_of_the_primary_legal_form,code_and_name_of_the_specific_legal_form,code_and_name_of_the_ownership_form,registration_authority,type_of_register_or_records,number_in_register_or_records,...,date_of_deletion_from_REGON,date_of_the_judgment_about_bankruptcy_declaration,date_of_insolvency_proceedings_ending,phone number,internal_phone_number,fax_number,email,website_address,PKD_code,PKD_name
0,932117189,8992321294,,AMW INVEST SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALNO...,1 - OSOBA PRAWNA,117 - SPÓŁKI Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ,112 - WŁASNOŚĆ PAŃSTWOWYCH OSÓB PRAWNYCH,"SĄD REJONOWY DLA M.ST.WARSZAWY W WARSZAWIE,XII...",REJESTR PRZEDSIĘBIORCÓW,0000003772,...,,,,,,,,,7111Z,DZIAŁALNOŚĆ W ZAKRESIE ARCHITEKTURY
1,015296322,5262675121,,"AMW TOWARZYSTWO BUDOWNICTWA SPOŁECZNEGO ""KWATE...",1 - OSOBA PRAWNA,117 - SPÓŁKI Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ,111 - WŁASNOŚĆ SKARBU PAŃSTWA,"SĄD REJONOWY DLA M.ST.WARSZAWY W WARSZAWIE,XII...",REJESTR PRZEDSIĘBIORCÓW,0000140528,...,,,,6876158,,6269150,KWATERA@WTBS-KWATERA.PL,,6832Z,ZARZĄDZANIE NIERUCHOMOŚCIAMI WYKONYWANE NA ZLE...
2,142989877,7010303102,,AMW SINEVIA SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALN...,1 - OSOBA PRAWNA,117 - SPÓŁKI Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ,111 - WŁASNOŚĆ SKARBU PAŃSTWA,"SĄD REJONOWY DLA M.ST.WARSZAWY W WARSZAWIE,XIV...",REJESTR PRZEDSIĘBIORCÓW,0000392868,...,,,,,,,,,4299Z,ROBOTY ZWIĄZANE Z BUDOWĄ POZOSTAŁYCH OBIEKTÓW ...
3,142990254,7010302456,,AMW REWITA SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALNO...,1 - OSOBA PRAWNA,117 - SPÓŁKI Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ,111 - WŁASNOŚĆ SKARBU PAŃSTWA,"SĄD REJONOWY DLA M.ST.WARSZAWY W WARSZAWIE,XIV...",REJESTR PRZEDSIĘBIORCÓW,0000394569,...,,,,,,,,,5520Z,OBIEKTY NOCLEGOWE TURYSTYCZNE I MIEJSCA KRÓTKO...
4,000617663,6570081317,,PRZEDSIĘBIORSTWO KOMUNIKACJI SAMOCHODOWEJ W KI...,1 - OSOBA PRAWNA,116 - SPÓŁKI AKCYJNE,111 - WŁASNOŚĆ SKARBU PAŃSTWA,"SĄD REJONOWY W KIELCACH, X WYDZIAŁ GOSPODARCZY...",REJESTR PRZEDSIĘBIORCÓW,0000047244,...,,2013-10-04,,413661500,,413660110,pks@pks.kielce.pl,pks.kielce.pl,4939Z,"POZOSTAŁY TRANSPORT LĄDOWY PASAŻERSKI, GDZIE I..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,000617700,6610200189,,PRZEDSIĘBIORSTWO KOMUNIKACJI SAMOCHODOWEJ W OS...,1 - OSOBA PRAWNA,116 - SPÓŁKI AKCYJNE,111 - WŁASNOŚĆ SKARBU PAŃSTWA,"SĄD REJONOWY W KIELCACH, X WYDZIAŁ GOSPODARCZY...",REJESTR PRZEDSIĘBIORCÓW,0000050306,...,,,,0412479791,,0412479789,zarzad@pksostrowiec.pl,www.pksostrowiec.pl,4939Z,"POZOSTAŁY TRANSPORT LĄDOWY PASAŻERSKI, GDZIE I..."
139,000616089,6340003271,,PRZEDSIĘBIORSTWO PRZEWOZU TOWARÓW POWSZECHNEJ ...,1 - OSOBA PRAWNA,116 - SPÓŁKI AKCYJNE,111 - WŁASNOŚĆ SKARBU PAŃSTWA,"SĄD REJONOWY KATOWICE-WSCHÓD W KATOWICACH, VII...",REJESTR PRZEDSIĘBIORCÓW,0000087670,...,,,,601422812,,,BIURO@PKSKATOWICE.COM.PL,,4520Z,"KONSERWACJA I NAPRAWA POJAZDÓW SAMOCHODOWYCH, ..."
140,000616050,5730206194,,PRZEDSIĘBIORSTWO KOMUNIKACJI SAMOCHODOWEJ CZĘS...,1 - OSOBA PRAWNA,116 - SPÓŁKI AKCYJNE,111 - WŁASNOŚĆ SKARBU PAŃSTWA,"SĄD REJONOWY W CZĘSTOCHOWIE, XVII WYDZIAŁ GOSP...",REJESTR PRZEDSIĘBIORCÓW,0000222666,...,,,,,,,,,4931Z,"TRANSPORT LĄDOWY PASAŻERSKI, MIEJSKI I PODMIEJSKI"
141,000617166,5250000127,,"PRZEDSIĘBIORSTWO KOMUNIKACJI SAMOCHODOWEJ ""POL...",1 - OSOBA PRAWNA,116 - SPÓŁKI AKCYJNE,111 - WŁASNOŚĆ SKARBU PAŃSTWA,"SĄD REJONOWY DLA M.ST.WARSZAWY W WARSZAWIE,XII...",REJESTR PRZEDSIĘBIORCÓW,0000376721,...,,,,8236200,,8236231,SEKRETARIAT@PKS.WARSZAWA.PL,,4939Z,"POZOSTAŁY TRANSPORT LĄDOWY PASAŻERSKI, GDZIE I..."


Saving downloaded informations in excel file.

In [11]:
df.to_excel('Data/Company_informations.xlsx', index = False)